## Summary

- Embeddings and continuous variables


## Reading Data

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

#increase width to 100%
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import numpy as np
import os
import sys
from pathlib import Path
from matplotlib import pyplot as plt
from collections import OrderedDict

from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, concatenate
from keras.layers import Input, Flatten, Dropout
from keras.optimizers import SGD, Adam
import seaborn as sns

#add fastai path
homedir = os.path.expanduser('~')
sys.path.append(os.path.join(homedir,'fastai'))
from fastai.structured import add_datepart

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.cross_validation import train_test_split

Using TensorFlow backend.
/home/achinta/miniconda3/envs/fastai/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### List, read the files and display summaries

In [2]:
datadir = Path(os.path.expanduser('~')).joinpath('data/kaggle/rossmann-store-sales')
files = [o for o in datadir.iterdir() if str(o).endswith('.csv')]

In [3]:
def read_data():
#     filenames = ['train','test','store']
#     train_df, test_df, store_df = [pd.read_csv(datadir.joinpath(filename + '.csv'),low_memory=False) for filename in filenames]
    train_df = pd.read_csv(datadir.joinpath('train.csv'),parse_dates=['Date'],low_memory=False)
    test_df = pd.read_csv(datadir.joinpath('test.csv'),parse_dates=['Date'],low_memory=False)
    store_df = pd.read_csv(datadir.joinpath('store.csv'))
    return train_df, test_df, store_df

train_df, test_df, store_df = read_data()

In [4]:
from IPython.display import HTML
for df in [train_df, test_df, store_df]:
    display(df.head(3))
    

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"


In [5]:
for df in [train_df, test_df, store_df]:
    display(df.describe())

,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday
count,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06
mean,5.584297e+02,3.998341e+00,5.773819e+03,6.331459e+02,8.301067e-01,3.815145e-01,1.786467e-01
std,3.219087e+02,1.997391e+00,3.849926e+03,4.644117e+02,3.755392e-01,4.857586e-01,3.830564e-01
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.800000e+02,2.000000e+00,3.727000e+03,4.050000e+02,1.000000e+00,0.000000e+00,0.000000e+00
50%,5.580000e+02,4.000000e+00,5.744000e+03,6.090000e+02,1.000000e+00,0.000000e+00,0.000000e+00
75%,8.380000e+02,6.000000e+00,7.856000e+03,8.370000e+02,1.000000e+00,1.000000e+00,0.000000e+00
max,1.115000e+03,7.000000e+00,4.155100e+04,7.388000e+03,1.000000e+00,1.000000e+00,1.000000e+00


,Id,Store,DayOfWeek,Open,Promo,SchoolHoliday
count,41088.000000,41088.000000,41088.000000,41077.000000,41088.000000,41088.000000
mean,20544.500000,555.899533,3.979167,0.854322,0.395833,0.443487
std,11861.228267,320.274496,2.015481,0.352787,0.489035,0.496802
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,10272.750000,279.750000,2.000000,1.000000,0.000000,0.000000
50%,20544.500000,553.500000,4.000000,1.000000,0.000000,0.000000
75%,30816.250000,832.250000,6.000000,1.000000,1.000000,1.000000
max,41088.000000,1115.000000,7.000000,1.000000,1.000000,1.000000


,Store,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear
count,1115.00000,1112.000000,761.000000,761.000000,1115.000000,571.000000,571.000000
mean,558.00000,5404.901079,7.224704,2008.668857,0.512108,23.595447,2011.763573
std,322.01708,7663.174720,3.212348,6.195983,0.500078,14.141984,1.674935
min,1.00000,20.000000,1.000000,1900.000000,0.000000,1.000000,2009.000000
25%,279.50000,717.500000,4.000000,2006.000000,0.000000,13.000000,2011.000000
50%,558.00000,2325.000000,8.000000,2010.000000,1.000000,22.000000,2012.000000
75%,836.50000,6882.500000,10.000000,2013.000000,1.000000,37.000000,2013.000000
max,1115.00000,75860.000000,12.000000,2015.000000,1.000000,50.000000,2015.000000


### Check for null values

In [6]:
def get_null_counts(df):
    null_counts = df.isnull().sum()

    #filter cols with counts > 0
    null_cols = null_counts[null_counts > 0]
    null_dict = null_cols.to_dict()
    return null_dict
    

def impute_nulls(df,cols=[]):
    if not cols:
        cols = get_null_counts(df)
    
    for col in cols:
        if df[col].dtype in [np.float64, np.float]:
            df[col] = df[col].fillna(df[col].mean())
        if df[col].dtype in ["O"]:
            df[col] = df[col].fillna("")
    return df
    

### Impute null values

In [7]:
#impute nulls in train_df
null_counts = get_null_counts(train_df)
print('nulls in train_df: ', null_counts)
train_df = impute_nulls(train_df,null_counts.keys())

#impute nulls in store_df
null_counts = get_null_counts(store_df)
print('nulls in store_df : ',null_counts)
store_df = impute_nulls(store_df,null_counts.keys())

#impute nulls in test_df
null_counts = get_null_counts(test_df)
print('nulls in test_df: ',null_counts)
test_df = impute_nulls(test_df,null_counts.keys())


nulls in train_df:  {}
nulls in store_df :  {'CompetitionDistance': 3, 'CompetitionOpenSinceMonth': 354, 'CompetitionOpenSinceYear': 354, 'Promo2SinceWeek': 544, 'Promo2SinceYear': 544, 'PromoInterval': 544}
nulls in test_df:  {'Open': 11}


## Understanding the data

## Adding Features

In [8]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

#creates a new column with the number of rows elapsed, since the column value was non-zero. 
#here we make use of cumsum to create a group
def add_elapsed(df,fieldname,prefix,drop_cumsum=True):
    df['cumsum'] = df[fieldname].cumsum()
    df[prefix+fieldname] = df.groupby('cumsum').cumcount()
    df[prefix+fieldname].fillna(0).astype(int)
    if drop_cumsum:
        df.drop('cumsum',axis=1)

#create scalers for continuous variables
def get_scaler(df, x_cols,y_col=None):
    scaler = MinMaxScaler()
    ndf = df[x_cols]
    if y_col:
        scaler.fit(ndf,df[y_col])
    else:
        scaler.fit(ndf)
    return scaler

#transform the continuous variable columns for which scalers are available
def scaler_transform(df, cols, scaler):
    ndf = df[cols]
    scaled_ndf = scaler.transform(ndf)
    df[cols] = scaled_ndf
    return df

#create label encoders for categorical variables
def get_encoders(train_df,test_df, cols):
    encoders = {}
    for col in cols:
        encoders[col] = LabelEncoder().fit(train_df[col].unique().tolist() + test_df[col].unique().tolist())
    return encoders

#transform teh categorical variables using previously created encoders
def encoders_transform(df, encoders={},cols=[]):
    if not cols:
        cols = encoders.keys()
    for col in cols:
        df[col] = encoders[col].transform(df[col])
    return df

#get embedding layers for categorical columns
def get_embedding_layers(encoders):
    embedding_layers = {}
    inputs = {}
    for col in encoders:
        encodings = encoders[col].__dict__['classes_']
        embedding_size = len(encodings)//2
        if embedding_size > 50:
            embedding_size = 50
        inputs[col] = Input(shape=(1,),name=col)
        embedding = Embedding(len(encodings),embedding_size,name='emb_' + col)(inputs[col])
        embedding_layers[col] = embedding

    return embedding_layers, inputs

def get_keras_data(df, cat_vars,contin_vars):
    x = {}
    for col in cat_vars:
        x[col] = np.array(df[col])
    
    #contin vars
    x['contin_df'] = df[contin_vars]
    return x

In [9]:
def add_features(train_df, test_df, store_df):
    add_datepart(train_df, "Date", drop=False)
    add_datepart(test_df, "Date", drop=False)
    
    #convert stateHoliday to boolean

    train_df.StateHoliday = train_df.StateHoliday!='0'
    test_df.StateHoliday = test_df.StateHoliday!='0'
    
    #Join store
    train_df = join_df(train_df, store_df, "Store")
    test_df = join_df(test_df, store_df, "Store")
    
    #remove the duplicate joined columns
    for df in [train_df, test_df]:
        for c in df.columns:
            if c.endswith('_y'):
                if c in df.columns:
                    df.drop(c, inplace=True, axis=1)
                    
    return (train_df, test_df)

train, test = add_features(train_df, test_df, store_df)

In [10]:
print(train.shape)
print(test.shape)
print('train columns not present in test: ',[o for o in train_df.columns if o not in test_df.columns])
print('test columns not present in train: ',[o for o in test_df.columns if o not in train_df.columns])

(1017209, 31)
(41088, 30)
train columns not present in test:  ['Sales', 'Customers']
test columns not present in train:  ['Id']


### Identify categorical and numeric columns

In [11]:
date_cols = ['Year','Month','Week','Day','Dayofweek','Dayofyear','Is_month_end','Is_month_start']
date_cols += ['Is_quarter_end','Is_quarter_start','Is_year_end','Is_year_start',]

cat_vars = date_cols + ['Store','StoreType','Assortment','Promo2','Open','Promo','StateHoliday','SchoolHoliday']
# cat_vars += ['PromoInterval']

contin_vars = ['CompetitionDistance','CompetitionOpenSinceMonth','CompetitionOpenSinceYear','Promo2SinceWeek','Promo2SinceYear']

# train_df.columns

### Scaling and Encoding

In [12]:
### scale numeric columns
scaler = get_scaler(train,contin_vars)
train = scaler_transform(train, contin_vars,scaler)
test = scaler_transform(test, contin_vars,scaler)

#scale y
y_scaler = MinMaxScaler()
train['y_scaled'] = y_scaler.fit_transform(train['Sales'].copy().values.reshape(-1,1))

#encode categorical columns
encoders = get_encoders(train, test, cat_vars)
train = encoders_transform(train, encoders)
test = encoders_transform(test, encoders)

/home/achinta/miniconda3/envs/fastai/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


### Validation Data

In [13]:
print(train_df.Date.max())
print(train_df.Date.min())

val_df = train[train_df.Date > '2015-07-15']
train2 = train[train_df.Date <= '2015-07-15']

print(val_df.shape)
print(train2.shape)

2015-07-31 00:00:00
2013-01-01 00:00:00
(17840, 32)
(999369, 32)


### Building the model

In [14]:
def get_model(lr=0.001,lr_decay=0):
    #embeddings for categorical columns
    embedding_dict, input_dict = get_embedding_layers(encoders)
    cat_features = concatenate([(embedding_dict[col]) for col in cat_vars])
    cat_flat = Flatten()(cat_features)
    
    contin_input = Input(shape=(len(contin_vars),),name="contin_df")
    contin_dense = Dense(32,activation='relu')(contin_input)

    x = concatenate([cat_flat,contin_dense])
    x = Dropout(0.2)(x)
    x = Dense(64,activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(64,activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    model = Model(inputs=[input_dict[col] for col in cat_vars] + [contin_input], outputs=[x])
    optimizer = Adam(lr=lr,decay=lr_decay)
    model.compile(loss='mean_squared_error',optimizer=optimizer,)
    return model

#changes to train and test df
if 'Customers' in train2.columns:
    train2.drop(labels='Customers',axis=1,inplace=True)
# if 'Sales' in train_df.columns:

model = get_model()

# model.summary()

/home/achinta/miniconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Training

In [15]:
x_train = get_keras_data(train2,cat_vars,contin_vars)
x_val   = get_keras_data(val_df,cat_vars,contin_vars)
# x_train.shape
hist = model.fit(x=x_train, y=train2['y_scaled'],epochs=2,validation_data=(x_val,val_df['y_scaled']))

Train on 999369 samples, validate on 17840 samples
Epoch 1/2
999369/999369 [==============================] - 125s 125us/step - loss: 0.0010 - val_loss: 9.4502e-04
Epoch 2/2
999369/999369 [==============================] - 120s 120us/step - loss: 6.5666e-04 - val_loss: 8.0264e-04


### Predictions

In [16]:
# model.predict()
x_test = get_keras_data(test,cat_vars)
preds = model.predict(x_test)
preds_scaled = y_scaler.inverse_transform(preds)

TypeError: get_keras_data() missing 1 required positional argument: 'contin_vars'

### Submission

In [ ]:
# preds_scaled
submit_df = pd.DataFrame({
    'Id': test['Id'],
    'Sales': preds_scaled[:,0]
})
submit_df.to_csv('rossman_submission.csv',index=False)
!kaggle competitions submit -c rossmann-store-sales -f rossman_submission.csv -m "only cat vars with embeddings"

Private LB score of 0.19994 (2667/3303)

#### adding continuous variables

Predict and submit